In [3]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data Preprocessing

In [6]:
train = pd.read_csv("C:/Users/Acer/Desktop/Kaggle/fake-news/train.csv")
test = pd.read_csv("C:/Users/Acer/Desktop/Kaggle/fake-news/test.csv")

In [7]:
train.shape

(20800, 5)

In [8]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [10]:
train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [42]:
## replacing the empty data values with an empty string
train = train.fillna('')
test = test.fillna('')

In [43]:
# merging author name and news title
train1 = train
test1 = test
train1['content'] = train['author'] + '' +  train['title']
test1['content'] = test['author'] + '' + test['title']

In [44]:
train1 = train1.drop('author', axis = 1)
train1 = train1.drop('title', axis = 1)
test1 = test1.drop('author', axis = 1)
test1 = test1.drop('title', axis = 1)

test1.head(2)

,id,text,content
0,20800,"PALO ALTO, Calif. — After years of scorning...",David StreitfeldSpecter of Trump Loosens Tongu...
1,20801,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists ne...


In [45]:
train1.head(2)

,id,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."


In [46]:
# Separating X and y
X = train1.drop('label', axis =1)
y = train1['label']

In [47]:
X.head(2)

,id,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,Ever get the feeling your life circles the rou...,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."


In [48]:
portstem = PorterStemmer()   ## this will reduce running to run, shopping to shop, etc.

In [49]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [portstem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [56]:
train['content'] = train['content'].apply(stemming)

In [57]:
print(train['content'])

0        darrel lucush dem aid even see comey letter ja...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exerci ba...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object


In [58]:
X  = train['content'].values
y = train['label'].values

In [59]:
X

array(['darrel lucush dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynnflynn hillari clinton big woman campu breitbart',
       'consortiumnew comwhi truth might get fire', ...,
       'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time',
       'alex ansarynato russia hold parallel exerci balkan',
       'david swansonwhat keep f aliv'], dtype=object)

In [60]:
len(X)

20800

In [61]:
y

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [62]:
y.shape

(20800,)

In [63]:
X.shape

(20800,)

## Vectorizer to convert textual data to numerical 

In [64]:
vec = TfidfVectorizer()
vec.fit(X)

TfidfVectorizer()

In [65]:
X = vec.transform(X)

In [66]:
print(X)

  (0, 26171)	0.28088379401596425
  (0, 22568)	0.2552336018069161
  (0, 14920)	0.43006226759639316
  (0, 14460)	0.29177259684200296
  (0, 12695)	0.24619727512767195
  (0, 7961)	0.23133661742488731
  (0, 6233)	0.2839932825877813
  (0, 5930)	0.35488202138141456
  (0, 4975)	0.2472595823572816
  (0, 4187)	0.3625320323150658
  (0, 572)	0.2694167078545385
  (1, 27744)	0.37204408227449065
  (1, 11235)	0.2434639969074374
  (1, 8704)	0.5300315426405939
  (1, 5877)	0.3206218375341294
  (1, 4739)	0.23523739119536424
  (1, 3838)	0.44601381123255807
  (1, 3260)	0.18800278047529806
  (1, 2602)	0.35911932371235855
  (2, 26066)	0.3665032495181434
  (2, 16256)	0.43295215406038445
  (2, 9384)	0.30743020569262086
  (2, 8499)	0.3411947414020896
  (2, 5204)	0.40440534260277944
  (2, 5089)	0.5511414848555652
  :	:
  (20797, 25609)	0.08234347981192172
  (20797, 25155)	0.3125002447493035
  (20797, 21934)	0.2494515865666979
  (20797, 20637)	0.2734270447609076
  (20797, 20353)	0.25042469546773877
  (20797, 17396

### Splitting into test and train data, note that this is splitting the original train data - not the test data given in the problem

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 11)

##  we will use Logistic Regression

In [68]:
lr = LogisticRegression()

In [69]:
lr.fit(X_train, y_train)

LogisticRegression()

In [70]:
y_test_pred = lr.predict(X_test)

In [71]:
y_train_pred = lr.predict(X_train)
accuracy_score(y_train, y_train_pred)

0.97

In [72]:
accuracy_score(y_test_pred, y_test)           #### nice

0.9571153846153846

## Now I will do the same preprocessing to real test data that I did to train data and then use the already created logistic regression model.

In [73]:
test.head(2)

,id,title,author,text,content
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",David StreitfeldSpecter of Trump Loosens Tongu...
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists ne...


In [74]:
test = test.fillna('')

In [75]:
test1.head(2)

,id,text,content
0,20800,"PALO ALTO, Calif. — After years of scorning...",David StreitfeldSpecter of Trump Loosens Tongu...
1,20801,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists ne...


In [77]:
X1  = train['content'].values
vec = TfidfVectorizer()
vec.fit(X1)

TfidfVectorizer()

In [79]:
X1 = vec.fit_transform(X1)

In [80]:
y1_pred = lr.predict(X1)

In [84]:
df = pd.DataFrame(y1_pred)
df.to_excel('output.xlsx', index=False)

## The end 